In [10]:
%matplotlib inline

import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

# Deep Neural Network
---
As neural network goes deeper and deeper, thorectically, it would perform better at least on training set, but in practice, the performance does not go better with deeper network. One major reason is `gradient vanishing` problem, in the backpropagation of a very deep network, the gradients at earlier layers would go to zero quickly that cause the learning process to be unbearably slow.

# ResNet
---
ResNet makes it possible, thoretically, to build infinite deeper neural network without impair the model perfermance, at least not getting worse. The major advanced structure in `Resnet` is called `skip connection`.

<img src="images/images/skip_connection_kiank.png" style="width:650px;height:200px;">

The left side is a plain network without `skip connection`, the right side is the one with a `skip connection`. 

`skip connection` allows activation values in earlier layers to be fast-forwarded to latter layers by addition. These has 2 advantages:

- In the forward propagation, the latter layer would at least have the performance of earlier layers:

$$ a^{[l + 2]} = g(z^{[l+2]} + a^{[l]})$$

Where $a^{[l]}$ is fast-forwarded from earlier layer. In the iteration process, as weights go to zero, $z$ would decrease to zero, in the worst scenario, even when $z$ totally lost its effect, activation value $a^{[l+2]}$ would at least have the value of $a^{[l]}$.

- In the backpropagation process, the gradient can be directly backpropagated to earlier layers so that there would be less multiplications in the backprogaction, thus accelerate the learning process.

## Identity Block
---
There are 2 sorts of `skip connections` in convolutional network, the first one is called `identity block`, shown as below:

<img src="images/images/idblock3_kiank.png" style="width:650px;height:150px;">

In this structure the activation value of earlier layer `x` is directly copied and added to the later layer before send to the `ReLU` activation.

Here're the individual steps.

First component of main path: 
- The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and its name should be `conv_name_base + '2a'`. Use 0 as the seed for the random initialization. 
- The first BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2a'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Second component of main path:
- The second CONV2D has $F_2$ filters of shape $(f,f)$ and a stride of (1,1). Its padding is "same" and its name should be `conv_name_base + '2b'`. Use 0 as the seed for the random initialization. 
- The second BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2b'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Third component of main path:
- The third CONV2D has $F_3$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and its name should be `conv_name_base + '2c'`. Use 0 as the seed for the random initialization. 
- The third BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2c'`. Note that there is no ReLU activation function in this component. 

Final step: 
- The shortcut and the input are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

In [27]:
def identity_block(X, f, nfilters, stage, block):
    """
    X has shape (m, n_H, n_W, n_C)
    f filter size of second conv + bn + relu
    """
    assert len(nfilters) == 3
    F1, F2, F3 = nfilters
    
    # note: for identity block, the last layer of input X need to has the same dimension of last filter
    # to enable Addition in the end
    assert F3 == X.shape[3]
    conv_base_name = 'conv_' + str(stage) + block
    bn_base_name = 'bn_' + str(stage) + block
    
    X_shortcut = X
    # first
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), 
               padding='valid', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2a')(X)
    X = Activation('relu')(X)
    
    # second
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), 
               padding='same', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2b')(X)
    X = Activation('relu')(X)
    
    # third
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), 
               padding='valid', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2a')(X)  # X shape: (m, n_H, n_W, F3)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [28]:
X = np.random.randn(3, 4, 4, 6)

X_output = identity_block(X, f=2, nfilters=[2, 4, 6], stage='1', block='a')

print('output shape', X_output.shape)

output shape (3, 4, 4, 6)
